<a href="https://colab.research.google.com/github/jogfx/ADL4/blob/main/ADL_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
!pip install lancedb sentence-transformers gradio nltk pyspellchecker -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 43.5 MB/s eta 0:00:00


In [48]:
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from spellchecker import SpellChecker
import nltk

In [49]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
spell = SpellChecker()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [50]:
import pandas as pd

# Load the CSV file directly from the provided GitHub URL
url = "https://raw.githubusercontent.com/jogfx/ADL4/main/Food%20Ingredients%20and%20Recipe%20Dataset%20with%20Image%20Name%20Mapping.csv"
df = pd.read_csv(url)


In [51]:
# Basic exploration
print(df.head())  # Display the first few rows to understand the structure
print(df.info())  # Get a summary of the dataset, including column names and data types

   Unnamed: 0                                              Title  \
0           0  Miso-Butter Roast Chicken With Acorn Squash Pa...   
1           1                    Crispy Salt and Pepper Potatoes   
2           2                        Thanksgiving Mac and Cheese   
3           3                 Italian Sausage and Bread Stuffing   
4           4                                       Newton's Law   

                                         Ingredients  \
0  ['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...   
1  ['2 large egg whites', '1 pound new potatoes (...   
2  ['1 cup evaporated milk', '1 cup whole milk', ...   
3  ['1 (¾- to 1-pound) round Italian loaf, cut in...   
4  ['1 teaspoon dark brown sugar', '1 teaspoon ho...   

                                        Instructions  \
0  Pat chicken dry with paper towels, season all ...   
1  Preheat oven to 400°F and line a rimmed baking...   
2  Place a rack in middle of oven; preheat to 400...   
3  Preheat oven to 350°F with 

In [52]:
df.rename(columns={'Unnamed: 0': 'id'}, inplace=True) # Rename the first column to 'id'

In [53]:
# Identify and remove duplicates
df.drop_duplicates(subset=['Title'], inplace=True) # Remove duplicates based on the 'Title' column

In [54]:
# Check for missing values and handle them accordingly
df.dropna(subset=['Title', 'Cleaned_Ingredients', 'Instructions', 'Image_Name'], inplace=True)

In [55]:
df['id'] = df['id'].astype('int64')

In [58]:
def tokenize_and_lowercase(text):
    return [token.lower() for token in word_tokenize(text)]

def remove_stop_words(tokens):
    return [word for word in tokens if word not in stop_words]

def lemmatize_words(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

#def correct_spelling(tokens):
    corrected_tokens = [spell.correction(token) for token in tokens]
    return corrected_tokens

def preprocess_text(text):
    tokens = tokenize_and_lowercase(text)
    tokens = remove_stop_words(tokens)
    tokens = lemmatize_words(tokens)
    #tokens = correct_spelling(tokens)
    return ' '.join(tokens)

In [59]:
text_columns = ['Title', 'Cleaned_Ingredients', 'Instructions']

for column in text_columns:
    df[column] = df[column].astype(str).apply(preprocess_text)

In [60]:
from sentence_transformers import SentenceTransformer

In [61]:
# Load SBERT model
model = SentenceTransformer('all-MiniLM-L12-v2')

In [62]:
vectors = model.encode(df.Cleaned_Ingredients.values.tolist(),
                       convert_to_numpy=True,
                       normalize_embeddings=True).tolist()

In [63]:
from lance.vector import vec_to_table
import lancedb
import numpy as np
import pyarrow as pa

In [64]:
db=lancedb.connect('DB/food.db')

In [65]:
table = db.create_table("food", df, mode='overwrite')

In [66]:
table.head().to_pandas()

,id,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients
0,0,miso-butter roast chicken acorn squash panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","pat chicken dry paper towel , season 2 tsp . s...",miso-butter-roast-chicken-acorn-squash-panzanella,"[ ' 1 ( 3½–4-lb . ) whole chicken ' , '2¾ tsp ..."
1,1,crispy salt pepper potato,"['2 large egg whites', '1 pound new potatoes (...",preheat oven 400°f line rimmed baking sheet pa...,crispy-salt-and-pepper-potatoes-dan-kluger,"[ ' 2 large egg white ' , ' 1 pound new potato..."
2,2,thanksgiving mac cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",place rack middle oven ; preheat 400° . bring ...,thanksgiving-mac-and-cheese-erick-williams,"[ ' 1 cup evaporated milk ' , ' 1 cup whole mi..."
3,3,italian sausage bread stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",preheat oven 350°f rack middle . generously bu...,italian-sausage-and-bread-stuffing-240559,"[ ' 1 ( ¾- 1-pound ) round italian loaf , cut ..."
4,4,newton 's law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",stir together brown sugar hot water cocktail s...,newtons-law-apple-bourbon-cocktail,"[ ' 1 teaspoon dark brown sugar ' , ' 1 teaspo..."


In [67]:
embeddings = vec_to_table(vectors)
embeddings = embeddings.append_column("id", pa.array(np.arange(len(table))+1))
embeddings.to_pandas().head()

,vector,id
0,"[-0.079072565, -0.017431559, -0.00510634, -0.0...",1
1,"[-0.094722755, -0.0024763518, -0.029430758, -0...",2
2,"[-0.10741216, -0.0038680558, 0.04530523, -0.06...",3
3,"[-0.10223243, -0.037277956, -0.01621729, -0.02...",4
4,"[-0.14619634, -0.042246494, 0.012004896, -0.03...",5


In [68]:
table.merge(embeddings, left_on="id")
table.head().to_pandas()

,id,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients,vector
0,0,miso-butter roast chicken acorn squash panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","pat chicken dry paper towel , season 2 tsp . s...",miso-butter-roast-chicken-acorn-squash-panzanella,"[ ' 1 ( 3½–4-lb . ) whole chicken ' , '2¾ tsp ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,crispy salt pepper potato,"['2 large egg whites', '1 pound new potatoes (...",preheat oven 400°f line rimmed baking sheet pa...,crispy-salt-and-pepper-potatoes-dan-kluger,"[ ' 2 large egg white ' , ' 1 pound new potato...","[-0.079072565, -0.017431559, -0.00510634, -0.0..."
2,2,thanksgiving mac cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",place rack middle oven ; preheat 400° . bring ...,thanksgiving-mac-and-cheese-erick-williams,"[ ' 1 cup evaporated milk ' , ' 1 cup whole mi...","[-0.094722755, -0.0024763518, -0.029430758, -0..."
3,3,italian sausage bread stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",preheat oven 350°f rack middle . generously bu...,italian-sausage-and-bread-stuffing-240559,"[ ' 1 ( ¾- 1-pound ) round italian loaf , cut ...","[-0.10741216, -0.0038680558, 0.04530523, -0.06..."
4,4,newton 's law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",stir together brown sugar hot water cocktail s...,newtons-law-apple-bourbon-cocktail,"[ ' 1 teaspoon dark brown sugar ' , ' 1 teaspo...","[-0.10223243, -0.037277956, -0.01621729, -0.02..."


In [122]:
def retrieve_similar_recipes(query_text, top_k=3):
    # Convert the query text to a vector
    query_vector = model.encode([query_text])

    # Open the table where your recipes are stored
    tbl = db.open_table("food")

    # Perform the vector search using euclidian distance
    search_results = tbl.search(query_vector[0]) \
        .metric("l2") \
        .limit(top_k) \
        .to_list()

    # Assuming search_results returns a list of documents, including the recipe details
    # Format the results to include Title and Instructions
    formatted_results = []
    for doc in search_results:
        title = doc['Title']
        ingredients = doc['Ingredients']
        instructions = doc['Instructions']
        formatted_result = f"Title: {title}\nIngredients: {ingredients}\nInstructions: {instructions}"
        formatted_results.append(formatted_result)

    return "\n\n".join(formatted_results)

In [123]:
# Example query for each technique
query_verbose = "I need a recipe for a vegetarian meal using ingredients like quinoa, black beans, and avocado, with a spicy kick to it."
query_concise = "tomato basil soup"
query_contextual = "Looking for a traditional Japanese dish to cook for a themed dinner party, something that can be made ahead of time."

# Retrieve recipes for each query
recipes_verbose = retrieve_similar_recipes(query_verbose)
recipes_concise = retrieve_similar_recipes(query_concise)
recipes_contextual = retrieve_similar_recipes(query_contextual)

In [124]:
# Evaluation: This part is subjective and may involve manual inspection or user feedback
print("Verbose Prompt Results:\n", recipes_verbose)
print("\n")
print("Concise Prompt Results:\n", recipes_concise)
print("\n")
print("Contextual Prompt Results:\n", recipes_contextual)

Verbose Prompt Results:
 Title: charred-peach panzanella pickled pepper vinaigrette
Ingredients: ['2 garlic cloves, finely grated', '½ cup sliced hot pickled peppers (such as banana peppers), plus 1 Tbsp. pickle brine', '1 Tbsp. stone-ground or whole grain mustard', '1½ tsp. kosher salt', '1½ lb. plum tomatoes (6–7 medium), sliced ½" thick', '8 oz. thick-cut bacon (about 6 slices), cut into 1½" pieces', '1 (10–12-oz.) loaf crusty multigrain bread, sliced ¾" thick', '3 large slightly firm peaches (about 1¼ lb.), preferably freestone, halved, pitted']
Instructions: whisk garlic , pickle brine , mustard , salt large bowl combine . add tomato pickled pepper gently toss combine . let sit , tossing occasionally , cook bacon , bread , peach . arrange bacon cold large cast-iron skillet set medium heat . cook , stirring occasionally , mostly crisp ( still soft , chewy spot ’ sort thing ) , 10 minute . transfer bacon paper towel drain . pour bacon fat small bowl ; set aside . wipe skillet . heat

In [126]:
import gradio as gr

def gradio_retrieve_similar_recipes(query):
    return retrieve_similar_recipes(query, top_k=3)

interface = gr.Interface(
    fn=gradio_retrieve_similar_recipes,
    inputs=gr.Textbox(lines=2, placeholder="Enter ingredients or recipe name..."),
    outputs=gr.Textbox(label="Top Similar Recipes"),
    title="Recipe Retrieval System",
    description="Enter ingredients or a recipe name to find top 3 similar recipes along with their instructions."
)

In [127]:
interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9ea5943bd13b51611a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
